## Step 1. 导入模块

In [89]:
import numpy as np
import pandas as pd
import plotly.graph_objects as go
import plotly.express as px
import jieba
from stylecloud import gen_stylecloud
from snownlp import SnowNLP
from IPython.display import Image # 用于在jupyter lab中显示本地图片
from PIL import Image, ImageDraw, ImageFont
import inspect

## Step 2. 数据分析可视化

### 2.1 高校评分数据概览

In [90]:
df =pd.read_excel('高校评分_1.xlsx')
df.head(10)

,school_type,nodeId,bizId,name,scorePersonCount,totalScore,scoreAvg,scoreDistribution,hottestComments
0,北京市高校,1420913,193510,淮安市高级职业技术学校,16,88,5.5,"{'2': 8, '4': 1, '8': 1, '10': 6}",[]
1,北京市高校,1420913,168680,中国伊斯兰教经学院,17,94,5.5,"{'2': 8, '4': 1, '6': 1, '8': 1, '10': 6}",[]
2,北京市高校,1420913,169361,中国人民大学外交学院,17,98,5.8,"{'2': 7, '4': 2, '6': 1, '10': 7}",[]
3,北京市高校,1420913,182660,中国科学院过程工程研究所,17,128,7.5,"{'2': 4, '6': 2, '8': 1, '10': 10}",[]
4,北京市高校,1420913,183280,中国科学院生态环境研究中心,17,112,6.6,"{'2': 4, '4': 1, '6': 4, '8': 2, '10': 6}",[]
5,北京市高校,1420913,183281,中国科学院国家空间科学中心,17,128,7.5,"{'2': 2, '4': 2, '6': 2, '8': 3, '10': 8}",[]
6,北京市高校,1420913,182666,中国科学院微生物研究所,17,112,6.6,"{'2': 5, '4': 1, '6': 2, '8': 2, '10': 7}",[]
7,北京市高校,1420913,183606,中国科学院国家纳米科学中心,17,120,7.1,"{'2': 3, '4': 1, '6': 4, '8': 2, '10': 7}",[]
8,北京市高校,1420913,183812,中国科学院空天信息创新研究院,17,112,6.6,"{'2': 4, '4': 2, '6': 3, '8': 1, '10': 7}",[]
9,北京市高校,1420913,182397,中国科学院数学与系统科学研究院,18,118,6.6,"{'2': 4, '4': 2, '6': 4, '8': 1, '10': 7}",[]


In [91]:
print("——" * 10)
print('数据集存在重复值个数：')
print(df.duplicated().sum())
print("——" * 10)
print('数据集缺失值情况：')
print(df.isna().sum())
print("——" * 10)
print('数据集各字段类型：')
print(df.dtypes)
print("——" * 10)
print('数据总体概览：')
print(df.info())

————————————————————
数据集存在重复值个数：
0
————————————————————
数据集缺失值情况：
school_type          0
nodeId               0
bizId                0
name                 0
scorePersonCount     0
totalScore           0
scoreAvg             0
scoreDistribution    0
hottestComments      0
dtype: int64
————————————————————
数据集各字段类型：
school_type           object
nodeId                 int64
bizId                  int64
name                  object
scorePersonCount       int64
totalScore             int64
scoreAvg             float64
scoreDistribution     object
hottestComments       object
dtype: object
————————————————————
数据总体概览：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 8586 entries, 0 to 8585
Data columns (total 9 columns):
 #   Column             Non-Null Count  Dtype  
---  ------             --------------  -----  
 0   school_type        8586 non-null   object 
 1   nodeId             8586 non-null   int64  
 2   bizId              8586 non-null   int64  
 3   name               8586 non-

### 2.2 虎扑评分人数Top20

In [92]:
df_sorted = df.sort_values(by='scorePersonCount', ascending=False).head(20)

In [93]:
from plotly.subplots import make_subplots

fig2 = make_subplots(specs=[[{"secondary_y":True}]])

fig2.add_trace(
    go.Scatter(x=df_sorted["name"],y=df_sorted["scoreAvg"],name="平均分"),secondary_y=True
)
fig2.add_trace(
    go.Bar(x=df_sorted["name"],y=df_sorted["scorePersonCount"],name="评分人数",)
)
fig2.update_yaxes(go.layout.YAxis(tickformat="",showgrid=False),secondary_y=True)
fig2.update_layout(go.Layout(title=go.layout.Title(text="虎扑评分人数Top20")),template="plotly_white",)

### 2.3 高校评论数据概览

In [94]:
df1 =pd.read_excel('高校评论_2.xlsx')
df1.head(10)

,school_type,nodeId,bizId,school_name,commentId,commentUserId,commentUserName,commentContent,lightCount,blackCount,score
0,北京市高校,1420913,6982,清华大学,908020575,17522295,Cauchy不懂中值,Goat,1,0,0
1,北京市高校,1420913,6982,清华大学,908394623,26047503,我不是登吹,那我也不上,1,0,0
2,北京市高校,1420913,6982,清华大学,908352136,98106072,笛亚,难道留着过年敬酒，就中国论资排辈那一套谁愿意，中科大不也有很多天才少年留学之后就留在国外了,1,0,0
3,北京市高校,1420913,6982,清华大学,908320027,25969403,AIO2000,很好，但我是铁红，只能这样了,3,0,8
4,北京市高校,1420913,6982,清华大学,907921615,25424391,理想之箭奥多姆,挺一般的,0,0,2
5,北京市高校,1420913,6982,清华大学,907961708,95435876,詹眉Lakers,你不上，我也不上了,8215,5,10
6,北京市高校,1420913,6982,清华大学,908000575,65788207,利物浦名宿贝麦卡利斯特,中国最杰出的学校,1,0,10
7,北京市高校,1420913,6982,清华大学,907985008,28645384,我名字长不长啊长不长,他们都不上，那我也不上,8967,5,0
8,北京市高校,1420913,6982,清华大学,908000363,43591427,三炮卫冕之日再改名,隔壁的表示，真有这么好还天天过来？,0,1,10
9,北京市高校,1420913,6982,清华大学,907928450,96076146,全力冲击保研,那还点五星（doge,3,0,10


In [95]:
print("——" * 10)
print('数据集存在重复值个数：')
print(df1.duplicated().sum())
print("——" * 10)
print('数据集缺失值情况：')
print(df1.isna().sum())
print("——" * 10)
print('数据集各字段类型：')
print(df1.dtypes)
print("——" * 10)
print('数据总体概览：')
print(df1.info())

————————————————————
数据集存在重复值个数：
0
————————————————————
数据集缺失值情况：
school_type        0
nodeId             0
bizId              0
school_name        0
commentId          0
commentUserId      0
commentUserName    0
commentContent     0
lightCount         0
blackCount         0
score              0
dtype: int64
————————————————————
数据集各字段类型：
school_type        object
nodeId              int64
bizId               int64
school_name        object
commentId           int64
commentUserId       int64
commentUserName    object
commentContent     object
lightCount          int64
blackCount          int64
score               int64
dtype: object
————————————————————
数据总体概览：
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 131409 entries, 0 to 131408
Data columns (total 11 columns):
 #   Column           Non-Null Count   Dtype 
---  ------           --------------   ----- 
 0   school_type      131409 non-null  object
 1   nodeId           131409 non-null  int64 
 2   bizId            131409 non-nu

### 2.4 虎扑-国内各区域评论数分布

In [96]:
school_type_couns = df1['school_type'].value_counts().reset_index()
school_type_couns.rename(columns={"index": "区域", "school_type": "评论数"}, inplace=True)
school_type_couns

,评论数,count
0,北京市高校,13803
1,江苏省高校,10674
2,河南省高校,9217
3,山东省高校,8982
4,广东省高校,8633
5,湖北省高校,5883
6,安徽省高校,5524
7,黑龙江省高校,5506
8,辽宁省高校,5467
9,四川省高校,4918


### 2.5 虎扑-国内各区域高校评论数分布

In [97]:
school_names_size = df1.groupby(['school_type', 'school_name']).size().sort_values(ascending=False).reset_index()

In [98]:
school_names_size.rename(columns={0: "评论数"}, inplace=True)

In [99]:
school_names_size.head(20)

,school_type,school_name,评论数
0,北京市高校,清华大学,6872
1,黑龙江省高校,哈尔滨工业大学,2518
2,北京市高校,北京大学,2481
3,四川省高校,四川大学,910
4,山东省高校,山东大学,883
5,河南省高校,周口职业技术学院,870
6,辽宁省高校,大连海事大学,813
7,上海市高校,上海交通大学,760
8,广东省高校,深圳大学,673
9,江苏省高校,南京大学,573


In [100]:

fig = px.treemap(school_names_size, 
                 path=["school_type", "school_name"],  # 指定层次结构，每一个层次都应该是category型的变量
                 values='评论数', # 需要聚合的列名
                 height = 1080,
                 width = 1200,
                 color_continuous_scale='Geyser',
                 color_continuous_midpoint=0 , # 颜色变化中间值设置为增长率=0
                 title="虎扑-国内各区域高校评论数分布",
                 template="plotly_white",
                )
fig.update_traces(textinfo='label+value',textfont = dict(size = 24)) # 显示企业名称和市值，字体24
fig.show()
